In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp 
import numpy as np

### TFBs overlap with non coding variants associated with gene expression 

In [2]:
#GWAS was re-run with TFBS filtered to focus on those with p-value < 0.01
#Get snp positions

genes = pd.read_csv('gene_positions.txt', sep = '\t', index_col = 'locus_tag')

variants = 'associations/asso_variant/'

# Get list of files
files = [file for file in os.listdir(variants) if file.endswith('.tsv')]

# Create an empty table
table = []

# Loop through each file
for file_name in files:
    # Remove the extension (.tsv) from the file name
    file_name_without_extension = file_name[:-4]
    
    # Split the file name by underscores
    parts = file_name_without_extension.split('_')
    
    # Extract locus tag (first part)
    locus_tag = parts[0]
    
    # Extract position (second part)
    position = parts[1]
    
    # Extract mutation type (remaining parts joined by underscore)
    mutation_type = '_'.join(parts[2:])
    
    # Append extracted information to the table
    table.append([locus_tag, position, mutation_type])

# Convert table to DataFrame
df = pd.DataFrame(table, columns=["Locus Tag", "Position", "Mutation Type"]).set_index('Locus Tag')
df = pd.merge(df, genes[['protein_id','strand','start','end']], left_index = True, right_index = True, how = 'left')
df['protein_id'] = 'cds-' + df['protein_id']
df = df.set_index('protein_id')

#deduce promoter sequence positions 

forward = df[df['strand']=='+'].copy()
forward['begin'] = forward.start.sub(200)
forward['stop'] = forward.start.add(30)

reverse = df[df['strand']=='-'].copy()
reverse['begin'] = reverse.end.sub(30)
reverse['stop'] = reverse.end.add(200)
reverse

promoter_pos = [forward, reverse]
df = pd.concat(promoter_pos, axis = 0)

df = df.drop(['start', 'end'], axis = 1).rename(columns = {'begin':'start', 'stop':'end'})
df.head(5)


,Position,Mutation Type,strand,start,end
protein_id,,,,,
cds-NP_414767.1,251965,A_T,+,251804,252034
cds-NP_414767.1,251995,GT_AG,+,251804,252034
cds-NP_414768.1,252142,C_T,+,252100,252330
cds-NP_414768.1,252208,A_G,+,252100,252330
cds-NP_414768.1,252148,G_A,+,252100,252330


In [3]:
asso_sites = [x.split('.tsv')[0] for x in os.listdir('associations/sig')]
len(asso_sites)

178

In [4]:
results = []

path = 'fimo_deg/'

for gene_name in asso_sites:
    gene_path = os.path.join(path, gene_name)
    if os.path.isdir(gene_path):
        with open(os.path.join(gene_path, 'fimo.tsv')) as file:
            
            if not any(line.strip() and not line.strip().startswith('#') for line in file):
                continue

        data = pd.read_csv(os.path.join(gene_path, 'fimo.tsv'), sep='\t')
        
        if data.empty:
            continue
        
        filtered_tf_data = data[data['q-value'] < 0.01].drop_duplicates(subset=['motif_id'], keep='first')
        
        if filtered_tf_data.empty:
            continue
        
        for index, tf_site in filtered_tf_data.iterrows():  # Focus on associated snps
            gene_info = df[df.index == gene_name]
            
            # Check if gene_info is empty
            if gene_info.empty:
                continue
            
            gene_info = gene_info.iloc[0]  # Assuming each gene occurs only once

            # Calculate TF site position relative to the whole genome
            
            if gene_info['strand'] == '+':  # Forward strand
                tf_site_position_start = gene_info['start'] + tf_site['start']
                tf_site_position_stop = tf_site_position_start + (tf_site['stop'] - tf_site['start'])
                
            else:  # Reverse strand
                tf_site_position_stop = gene_info['end'] - tf_site['start']
                tf_site_position_start = tf_site_position_stop - (tf_site['stop'] - tf_site['start'])

            results.append({
                'Gene': gene_name,
                'SNP_position': gene_info['Position'],
                'TF_binding_site': tf_site['motif_id'],
                'TF_site_start_position': tf_site_position_start,
                'TF_site_stop_position': tf_site_position_stop,
                'Gene_start_position': gene_info['start'],
                'Gene_end_position': gene_info['end'],
                'Strand': gene_info['strand']
            })

# Convert the list of dictionaries to a DataFrame
results_df = pd.DataFrame(results)
results_df

,Gene,SNP_position,TF_binding_site,TF_site_start_position,TF_site_stop_position,Gene_start_position,Gene_end_position,Strand
0,cds-NP_416434.1,2003722,YTAWTSVCSSMAWTAAC,2003770.0,2003786.0,2003671,2003901,+
1,cds-NP_416434.1,2003722,CATGAAAADHHTTCVCTT,2003856.0,2003873.0,2003671,2003901,+
2,cds-NP_416434.1,2003722,RMMWTTCRWWWTRWWWCKWWWK,2003855.0,2003876.0,2003671,2003901,+
3,cds-NP_416025.1,1592486,WTTTATCCYSTADVGCGGATAAR,1592529.0,1592551.0,1592412,1592642,-
4,cds-NP_416025.1,1592486,TATCCBSTADMGCGGATA,1592531.0,1592548.0,1592412,1592642,-
...,...,...,...,...,...,...,...,...
67,cds-NP_418319.2,4073542,TRGCAVWWWDKGYHAWW,4073594.0,4073610.0,4073538,4073768,+
68,cds-NP_418319.2,4073542,ABTTGTAAT,4073644.0,4073652.0,4073538,4073768,+
69,cds-NP_414768.1,252142,AWHMTTAABRWWWNCTTAA,252244.0,252262.0,252100,252330,+
70,cds-NP_416075.1,1641592,ATYSMTDARNT,1641559.0,1641569.0,1641524,1641754,-


In [5]:
#check if mapping was successful
results_df['comp_TFBS_length'] = results_df['TF_site_stop_position'] - results_df['TF_site_start_position'] + 1
results_df['actual_TFBS_length'] = [len(l) for l in results_df['TF_binding_site']]
results_df['SNP_position'] = results_df['SNP_position'].astype(int)
results_df['TF_site_start_position'] = results_df['TF_site_start_position'].astype(int)
results_df['TF_site_stop_position'] = results_df['TF_site_stop_position'].astype(int)
results_df

,Gene,SNP_position,TF_binding_site,TF_site_start_position,TF_site_stop_position,Gene_start_position,Gene_end_position,Strand,comp_TFBS_length,actual_TFBS_length
0,cds-NP_416434.1,2003722,YTAWTSVCSSMAWTAAC,2003770,2003786,2003671,2003901,+,17.0,17
1,cds-NP_416434.1,2003722,CATGAAAADHHTTCVCTT,2003856,2003873,2003671,2003901,+,18.0,18
2,cds-NP_416434.1,2003722,RMMWTTCRWWWTRWWWCKWWWK,2003855,2003876,2003671,2003901,+,22.0,22
3,cds-NP_416025.1,1592486,WTTTATCCYSTADVGCGGATAAR,1592529,1592551,1592412,1592642,-,23.0,23
4,cds-NP_416025.1,1592486,TATCCBSTADMGCGGATA,1592531,1592548,1592412,1592642,-,18.0,18
...,...,...,...,...,...,...,...,...,...,...
67,cds-NP_418319.2,4073542,TRGCAVWWWDKGYHAWW,4073594,4073610,4073538,4073768,+,17.0,17
68,cds-NP_418319.2,4073542,ABTTGTAAT,4073644,4073652,4073538,4073768,+,9.0,9
69,cds-NP_414768.1,252142,AWHMTTAABRWWWNCTTAA,252244,252262,252100,252330,+,19.0,19
70,cds-NP_416075.1,1641592,ATYSMTDARNT,1641559,1641569,1641524,1641754,-,11.0,11


In [6]:
#which gene's  nCVs overlap with TFBs
overlap = results_df[(results_df['SNP_position'] >= results_df['TF_site_start_position']) & 
                     (results_df['SNP_position'] <= results_df['TF_site_stop_position'])]
len(overlap['SNP_position'].unique())

4

In [7]:
overlap

,Gene,SNP_position,TF_binding_site,TF_site_start_position,TF_site_stop_position,Gene_start_position,Gene_end_position,Strand,comp_TFBS_length,actual_TFBS_length
6,cds-NP_416025.1,1592486,KWWYWWAANTTWWWWAANT,1592477,1592495,1592412,1592642,-,19.0,19
22,cds-NP_418309.1,4061088,AYRGMNKDWHHDWCYATA,4061087,4061104,4060964,4061194,+,18.0,18
30,cds-NP_416861.1,2473971,TMMWWCGRWAWAWMCKRWTT,2473969,2473988,2473831,2474061,+,20.0,20
38,cds-NP_418513.4,4313060,AAAATTGTVCANNNADDMBHWAHTGTACAABTTKYBT,4313057,4313093,4312961,4313191,-,37.0,37


In [8]:
overlap = results_df[(results_df['TF_site_start_position'] <= results_df['SNP_position']) & 
                     (results_df['TF_site_stop_position'] > results_df['SNP_position'])]

num_overlapping_tfbs = len(overlap)
num_unique_snps_with_tfbs = len(overlap['SNP_position'].unique())

In [9]:
num_unique_snps_with_tfbs

4

In [10]:
overlap['ids'] = [i.split('-')[1] for i in overlap['Gene']]
overlap = overlap.set_index('ids')
overlap_gene = pd.merge(genes[['gene_name','protein_id']], overlap, left_on = 'protein_id', right_index = True,how ='right')
overlap_gene

/var/folders/ys/t0s5987940jf4x94c3v464w00000gn/T/ipykernel_10797/1714730405.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  overlap['ids'] = [i.split('-')[1] for i in overlap['Gene']]


,gene_name,protein_id,Gene,SNP_position,TF_binding_site,TF_site_start_position,TF_site_stop_position,Gene_start_position,Gene_end_position,Strand,comp_TFBS_length,actual_TFBS_length
locus_tag,,,,,,,,,,,,
b1508,hipB,NP_416025.1,cds-NP_416025.1,1592486,KWWYWWAANTTWWWWAANT,1592477,1592495,1592412,1592642,-,19.0,19
b3873,yihM,NP_418309.1,cds-NP_418309.1,4061088,AYRGMNKDWHHDWCYATA,4061087,4061104,4060964,4061194,+,18.0,18
b2360,yfdQ,NP_416861.1,cds-NP_416861.1,2473971,TMMWWCGRWAWAWMCKRWTT,2473969,2473988,2473831,2474061,+,20.0,20
b4089,alsR,NP_418513.4,cds-NP_418513.4,4313060,AAAATTGTVCANNNADDMBHWAHTGTACAABTTKYBT,4313057,4313093,4312961,4313191,-,37.0,37


In [11]:
tf_overlap_counts = overlap.groupby('SNP_position')['TF_binding_site'].nunique().reset_index()
tf_overlap_counts

,SNP_position,TF_binding_site
0,1592486,1
1,2473971,1
2,4061088,1
3,4313060,1


In [12]:
# # Sample data: replace this with your actual data
# data = {
#     "gene_name": ["b1508", "b3873", "b2360", "b4089"],
#     "SNP_position": [1592486, 4061088, 2473971, 4313060],
#     "TF_site_start_position": [1592477, 4061087, 2473969, 4313057],
#     "TF_site_stop_position": [1592495, 4061104, 2473988, 4313093],
#     "Gene_start_position": [1592412, 4060964, 2473831, 4312961],  # Example Gene Start Positions
# }

# Create a DataFrame from the data
df = overlap_gene.copy()

# Initialize new columns for relative positions
df['SNP_relative_position'] = None
df['TFBS_relative_start'] = None
df['TFBS_relative_stop'] = None

# Loop through each gene and calculate relative positions
for index, row in df.iterrows():
    sequence_start = row['Gene_start_position']  # Get the Gene start position for the current gene

    # Calculate relative positions for the current gene
    df.at[index, 'SNP_relative_position'] = row['SNP_position'] - sequence_start
    df.at[index, 'TFBS_relative_start'] = row['TF_site_start_position'] - sequence_start
    df.at[index, 'TFBS_relative_stop'] = row['TF_site_stop_position'] - sequence_start

          gene_name SNP_relative_position TFBS_relative_start  \
locus_tag                                                       
b1508          hipB                    74                  65   
b3873          yihM                   124                 123   
b2360          yfdQ                   140                 138   
b4089          alsR                    99                  96   

          TFBS_relative_stop  
locus_tag                     
b1508                     83  
b3873                    140  
b2360                    157  
b4089                    132  


In [13]:
df

,gene_name,protein_id,Gene,SNP_position,TF_binding_site,TF_site_start_position,TF_site_stop_position,Gene_start_position,Gene_end_position,Strand,comp_TFBS_length,actual_TFBS_length,SNP_relative_position,TFBS_relative_start,TFBS_relative_stop
locus_tag,,,,,,,,,,,,,,,
b1508,hipB,NP_416025.1,cds-NP_416025.1,1592486,KWWYWWAANTTWWWWAANT,1592477,1592495,1592412,1592642,-,19.0,19,74,65,83
b3873,yihM,NP_418309.1,cds-NP_418309.1,4061088,AYRGMNKDWHHDWCYATA,4061087,4061104,4060964,4061194,+,18.0,18,124,123,140
b2360,yfdQ,NP_416861.1,cds-NP_416861.1,2473971,TMMWWCGRWAWAWMCKRWTT,2473969,2473988,2473831,2474061,+,20.0,20,140,138,157
b4089,alsR,NP_418513.4,cds-NP_418513.4,4313060,AAAATTGTVCANNNADDMBHWAHTGTACAABTTKYBT,4313057,4313093,4312961,4313191,-,37.0,37,99,96,132
